# Password storage


## Password Storage using PyNaCl

Password hashing are based on the idea of iterating a hash function many times on a combination of the password and a random salt, which is stored along with the hash.
The example shows that hashing the password multiple times results in different hashes, due to the random salt.
The default password storage mechanism in *libsodium* is *argon2id*, but nacl.pwhash also exposes *argon2i* and *script*.

From https://argon2.online/, https://github.com/P-H-C/phc-winner-argon2:

"Argon2i, Argon2d, and Argon2id are parametrized by:

* A time cost, which defines the amount of computation realized and therefore the execution time, given in number of iterations
* A memory cost, which defines the memory usage, given in kibibytes
* A parallelism degree, which defines the number of parallel threads"


* Argon2d maximizes resistance to GPU cracking attacks. It accesses the memory array in a password dependent order, which reduces the possibility of time–memory trade-off (TMTO) attacks, but introduces possible side-channel attacks.
* Argon2i is optimized to resist side-channel attacks. It accesses the memory array in a password independent order.
* Argon2id is a hybrid version. It follows the Argon2i approach for the first pass over memory and the Argon2d approach for subsequent passes. The Internet draft[4] recommends using Argon2id except when there are reasons to prefer one of the other two modes.


In [66]:
import nacl.pwhash as pwhash

password = b'correct horse battery staple' 

for i in range(3):
    print(password ) # todo replace by hashing the password
    


b'correct horse battery staple'
b'correct horse battery staple'
b'correct horse battery staple'


In the output:
* `argon2id` — the variant of Argon2 being used.
* `v=19` — the version of Argon2 being used.
* `m=65536,t=2,p=1` — the memory (m), iterations (t) and parallelism (p) parameters being used.
* `$(...)$(...)` — the base64-encoded salt, using standard base64-encoding and no padding along with the base64-encoded hashed password (derived key), using standard base64-encoding and no padding. The salt and password are separated by the symbol $.




In [67]:


# todo verify password





If the verification fails, an *InvalidkeyError* exception is raised:



In [68]:
wrong_pass = b'A really strong password'

try:
    print(password )# todo verify password
    
except pwhash.InvalidkeyError:
    print ("Wrong password!!")


b'correct horse battery staple'


### Why not use SHA or MD5 for password storage

Password hash should take the following considerations:
**Collision attacks**: exclude MD5 (128-bit hash size)
**Lookup Tables**: the solution is to add a random salt, which argon2id automatically does
**Key-stretching algorithms**: slow down the password verification, to slow down brute-force attacks

Let's compare the speed between a SHA-256 hash verification and the argon2id:

In [69]:
import nacl.encoding
import nacl.hash
from nacl.bindings.utils import sodium_memcmp

HASHER = nacl.hash.sha256
digest = HASHER(password, encoder=nacl.encoding.HexEncoder)



In [70]:
%%time
for lp in range(50):
    digest2 = HASHER(password, encoder=nacl.encoding.HexEncoder)
    sodium_memcmp(digest, digest2)

CPU times: user 832 µs, sys: 972 µs, total: 1.8 ms
Wall time: 1.03 ms


In [71]:
%%time
for lp in range(50):
    pwhash.verify(hashed, password)

CPU times: user 3.15 s, sys: 714 ms, total: 3.87 s
Wall time: 3.88 s




# Message Authentication Codes





In [9]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import cmac, hashes, hmac
from cryptography.hazmat.primitives.ciphers import algorithms

key = b'$C&F)J@NcRfUjXnZ' # 128-bit key for AES

#TODO compute the MAC


In [10]:
# todo verify the MAC



In [11]:
h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
h.update(b"message to hash")
h.finalize()

b'\xb0~*\x9a\x96\xd3=\xc0\xa9(\x02\x97Kq\xb4\x8brb\xb8\xc9M2\xd4a}Em\xa7\xec.g\x12'

In [12]:
h = hmac.HMAC(key, hashes.SHA256(), backend=default_backend())
h.update(b"message to hash")
h.verify(b"an incorrect signature")

InvalidSignature: Signature did not match digest.

# Authenticated encryption


Authenticated encryption (AE) and authenticated encryption with associated data (AEAD) are forms of encryption which simultaneously assure the confidentiality and authenticity of data.
These attributes are provided under a single, easy to use programming interface.


![](res/aead.png)


In [ ]:
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

plaintext = b"a secret and encrypted message"

associated_data = b"authenticated but unencrypted data"

key = AESGCM.generate_key(bit_length=256)
aesgcm = AESGCM(key)



In [ ]:


print(ciphertext)

In [ ]:
aesgcm.decrypt(nonce, ciphertext, associated_data)

In [ ]:
aesgcm.decrypt(nonce, ciphertext, b"altered associated data")